In [1]:
import os
import cv2
from PIL import Image
import numpy as np
import pickle as pkl
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
import torchvision
import torchvision.transforms as transforms
from processer import equalize, zmIceColor, extractGreen
from dataset import PlantSeedDataset

In [2]:
datasets_path = "../dataset/classifer/train"
    
transform = transforms.Compose([
        transforms.Resize((256, 256)),
        np.asarray,
        transforms.Lambda(lambda x: cv2.cvtColor(x, cv2.COLOR_RGB2BGR))
        ])
dataset = PlantSeedDataset(datasets_path, transform=transform)

In [25]:
len(dataset.labels)

4440

In [5]:
with open('all_feature_bow.pkl', 'rb') as f:
    all_feature_bow = pkl.load(f)

In [20]:
len(all_feature_bow)

4440

In [29]:
index = []
for i in range(len(all_feature_bow)):
    if type(all_feature_bow[i]) == type(None):
        index.append(i)

In [32]:
for i in index:
    del dataset.labels[i]

In [23]:
new_list = [x for x in all_feature_bow if type(x) != type(None)]

In [35]:
all_feature_bow = new_list

In [41]:
from sklearn.model_selection import train_test_split
X_train,X_test, y_train, y_test =train_test_split(all_feature_bow, dataset.labels, test_size=0.2, random_state=0)

In [37]:
len(X_train), len(y_train), len(X_test), len(y_test)

(3550, 3550, 888, 888)

In [44]:
for i in range(len(X_train)):
    X_train[i] = X_train[i].reshape(-1)

In [45]:
X_train[0].shape

(100,)

In [11]:
from xgboost import XGBClassifier
import xgboost

In [61]:
model_xgb = XGBClassifier(learning_rate=0.05, 
                          objective='binary:logistic',
                          booster="gbtree",
                          num_class=12, 
                          n_estimators=1000,
                          tree_method="hist",
                          device="cuda",
                          use_label_encoder=False)

In [62]:
model_xgb.fit(X_train, y_train)

XGBClassifier(base_score=None, booster='gbtree', callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device='cuda', early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.05, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=1000, n_jobs=None, num_class=12,
              num_parallel_tree=None, ...)

In [64]:
model_xgb.save_model("xgb.json")

In [53]:
for i in range(len(X_test)):
    X_test[i] = X_test[i].reshape(-1)

In [63]:
import sklearn.metrics
y_pred = model_xgb.predict(X_test)
sklearn.metrics.f1_score(y_test, y_pred, average='micro')

0.4391891891891892